# Covid vs India 
One of the biggest question I have been wondering is , with every adult allowed now to register, Where are vaccines slots available in india ? How can I make this decision programmitcally because trying to find information on government website is rather "sisyphus task".

So with rather self involed goal in mind, We would using publicly avaialbe data try to answer following questions

1. Latest vaccine slots per district and State in India
2. Top 5 States Running behind on schedule
3. Top 5 States Running ahead of everyone else.
4. When can I latest find slots in Delhi, Bombay, Chennai and Bangalore for people between 18-45


- toc: false
- branch: master
- badges: false,
- comments: true,
- categories: [vaccine, covid, jupyter, python],
- image: images/statistics.png,
- hide: false

Thanks to https://github.com/bhattbhavesh91/cowin-vaccination-slot-availability/blob/main/cowin-api-availability.ipynb for doing actual work, I picked up loads of stuff from there.


In [105]:
!{sys.executable} -m pip install --user install requests
!{sys.executable} -m pip install --user install altair
!{sys.executable} -m pip install --user install pandas
!{sys.executable} -m pip install --user install geopandas
#hide

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


Get All libraries in place

In [209]:
import geopandas as gpd
import pandas as pd
import requests
import json
from collections import defaultdict
from dataclasses import dataclass, asdict
import datetime
from typing import List
import uuid

Lets make a data class to store our geographical and vaccination data

In [210]:
@dataclass
class District:
    district_id:str = None
    name:str = None
    state_id:int = None


@dataclass
class Session:
    session_id:str = None
    date:datetime.datetime = None
    query_date:datetime.datetime= None
    available_capacity:int = None
    min_age_limit:int = None
    vaccine:str = None
    center_id:str = None
    district_id:str = None
        
@dataclass
class Center:
    center_id:str = None
    name:str = None
    state_name:str = None
    district_name:str = None
    block_name:str = None
    lat:int = None
    lng:int = None
    from_hour:datetime.datetime = None
    to_hour:datetime.datetime = None
    fee_type:str = None
    district_id:str = None

@dataclass
class NoSlotAvailable:
    district_id:str = None
    date:datetime.datetime = None

Lets call the API to get the populate geographical data

In [135]:
MOZILLA_HEADER = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
GET_DISTRICT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}"
GET_APOINTMENT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}"

DISTRICTS = defaultdict(District)
CENTERS = defaultdict(Center)
SESSIONS = defaultdict(Session)
NO_SLOT_AVAILABLE = []

MAX_NUMBER_OF_STATES = 40

for state_code in range(1, MAX_NUMBER_OF_STATES):
    headers = {'User-Agent': MOZILLA_HEADER}
    response = requests.get(GET_DISTRICT_DATA_API_URL.format(state_code), headers=headers)
    districts_data = json.loads(response.content)
    for district in districts_data['districts']:
        district_name = district['district_name']
        district_id = district['district_id']
        district = District(name = district_name, district_id = district_id , state_id=state_code) 
        DISTRICTS[district_id] = district

In [109]:
DISTRICT_DF = pd.DataFrame.from_dict([asdict(district) for district in DISTRICTS.values()])

Lets now call actual api to get slots 

In [ ]:
MAX_DAYS = 2

def get_data_fiften_days_in_future_from_today():
    base = datetime.datetime.today()
    date_list = [base + datetime.timedelta(days=x) for x in range(MAX_DAYS)]
    return [x.strftime("%d-%m-%Y") for x in date_list]

for district_id in DISTRICTS.keys():
    for slot_date in get_data_fiften_days_in_future_from_today():
        URL = GET_APOINTMENT_DATA_API_URL.format(district_id, slot_date)
        response = requests.get(URL)
        if response.ok:
            resp_json = response.json()
            if resp_json["centers"]:
                for center in resp_json["centers"]:
                    center_uuid = str(uuid.uuid4())
                    center_id = center["center_id"]
                    center_name = center["name"]
                    CENTERS[center_uuid] = Center(center_id=center_id,
                                                name=center_name,
                                                lat=center["lat"],
                                                lng=center["long"],
                                                from_hour=center["from"],
                                                to_hour=center["to"],
                                                district_id=district_id,
                                                state_name=center["state_name"],
                                                district_name=center["district_name"],
                                                block_name=center["block_name"],
                                                fee_type=center["fee_type"])
                    for session in center["sessions"]:
                        session_id = session["session_id"]
                        SESSIONS[session_id] = Session(session_id=session_id,
                                                       date=session["date"],
                                                       query_date=slot_date,
                                                       available_capacity=session["available_capacity"],
                                                       min_age_limit=session["min_age_limit"],
                                                       vaccine=session["vaccine"],
                                                       district_id=district_id,
                                                       center_id=center_id)
            else:
                NO_SLOT_AVAILABLE.append(NoSlotAvailable(district_id=district_id, date=slot_date))
                print("No slot on {} in district {}".format(slot_date, district_id))

No slot on 01-05-2021 in district 22
No slot on 02-05-2021 in district 22
No slot on 02-05-2021 in district 20
No slot on 02-05-2021 in district 25
No slot on 02-05-2021 in district 42
No slot on 02-05-2021 in district 24
No slot on 02-05-2021 in district 27
No slot on 01-05-2021 in district 21
No slot on 02-05-2021 in district 21
No slot on 01-05-2021 in district 33
No slot on 02-05-2021 in district 33
No slot on 01-05-2021 in district 29
No slot on 02-05-2021 in district 29
No slot on 02-05-2021 in district 40
No slot on 02-05-2021 in district 31
No slot on 01-05-2021 in district 18
No slot on 02-05-2021 in district 18
No slot on 02-05-2021 in district 36
No slot on 01-05-2021 in district 39
No slot on 02-05-2021 in district 39
No slot on 02-05-2021 in district 35
No slot on 02-05-2021 in district 37
No slot on 01-05-2021 in district 26
No slot on 02-05-2021 in district 26
No slot on 01-05-2021 in district 34
No slot on 02-05-2021 in district 34
No slot on 02-05-2021 in district 28
N

In [140]:
CENTER_DF = pd.DataFrame.from_dict([asdict(district) for district in CENTERS.values()])
SESSION_DF = pd.DataFrame.from_dict([asdict(session) for session in SESSIONS.values()])
NO_SLOT_AVAILABLE_DF = pd.DataFrame.from_dict([asdict(no_slot_available) for no_slot_available in NO_SLOT_AVAILABLE])

In [141]:
NO_SLOT_AVAILABLE_DF.head()

,district_id,date
0,26,01-05-2021
1,223,01-05-2021
2,304,01-05-2021
3,432,01-05-2021
4,431,01-05-2021


In [128]:
SESSION_DF.head()

,session_id,date,available_capacity,min_age_limit,vaccine,center_id,district_id
0,6ae7251c-9bc9-4475-bb2c-fda9af59947d,01-05-2021,50.0,45,,552109,3
1,1dbe9bab-a8b5-4454-87f8-e46df9b63f59,03-05-2021,50.0,45,,552109,3
2,9560db82-19e4-48e7-b93d-96ac43c71e2d,04-05-2021,48.0,45,,552109,3
3,b77bdec0-ba0e-404f-8da5-2ba36a7dec0f,05-05-2021,50.0,45,,552109,3
4,9417c021-66b2-4b10-8540-7ba91fef27f0,06-05-2021,50.0,45,,552109,3


In [129]:
CENTER_DF.head()

,center_id,name,state_name,district_name,block_name,lat,lng,from_hour,to_hour,fee_type,district_id
0,552109,Campbellbay PHC,Andaman and Nicobar Islands,Nicobar,Campbell Bay,7.0,93.0,09:00:00,17:00:00,Free,3
1,570779,BJR Hospital,Andaman and Nicobar Islands,Nicobar,Car Nicobar,9.0,92.0,09:00:00,17:00:00,Free,3
2,552108,Nancowry CHC,Andaman and Nicobar Islands,Nicobar,Nancowry,7.0,93.0,09:00:00,17:00:00,Free,3
3,639986,SMC 37 WING,Andaman and Nicobar Islands,Nicobar,Car Nicobar,9.0,92.0,09:00:00,18:00:00,Free,3
4,678139,TUGAPUR PHC,Andaman and Nicobar Islands,North and Middle Andaman,Mayabunder,12.0,92.0,09:00:00,17:00:00,Free,1


In [130]:
DISTRICT_DF.head()

,district_id,name,state_id
0,3,Nicobar,1
1,1,North and Middle Andaman,1
2,2,South Andaman,1
3,9,Anantapur,2
4,10,Chittoor,2


In [131]:
session_center_merged_df = pd.merge(SESSION_DF, CENTER_DF, on="center_id")
session_center_district_df = pd.merge(session_center_merged_df, DISTRICT_DF, left_on='district_id_x', right_on="district_id")

In [147]:
session_center_district_df.to_csv("vaccination_slot_data")
session_center_district_df

,session_id,date,available_capacity,min_age_limit,vaccine,center_id,district_id_x,name_x,state_name,district_name,block_name,lat,lng,from_hour,to_hour,fee_type,district_id_y,district_id,name_y,state_id
0,6ae7251c-9bc9-4475-bb2c-fda9af59947d,01-05-2021,50.0,45,,552109,3,Campbellbay PHC,Andaman and Nicobar Islands,Nicobar,Campbell Bay,7.0,93.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
1,1dbe9bab-a8b5-4454-87f8-e46df9b63f59,03-05-2021,50.0,45,,552109,3,Campbellbay PHC,Andaman and Nicobar Islands,Nicobar,Campbell Bay,7.0,93.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
2,9560db82-19e4-48e7-b93d-96ac43c71e2d,04-05-2021,48.0,45,,552109,3,Campbellbay PHC,Andaman and Nicobar Islands,Nicobar,Campbell Bay,7.0,93.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
3,b77bdec0-ba0e-404f-8da5-2ba36a7dec0f,05-05-2021,50.0,45,,552109,3,Campbellbay PHC,Andaman and Nicobar Islands,Nicobar,Campbell Bay,7.0,93.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
4,9417c021-66b2-4b10-8540-7ba91fef27f0,06-05-2021,50.0,45,,552109,3,Campbellbay PHC,Andaman and Nicobar Islands,Nicobar,Campbell Bay,7.0,93.0,09:00:00,17:00:00,Free,3,3,Nicobar,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148109,5e58fe48-9806-45b9-9d0c-be7ad987bee8,01-05-2021,10.0,45,,551644,139,PHC Vanakbara,Daman and Diu,Diu,Diu,20.0,70.0,09:00:00,17:00:00,Free,139,139,Diu,37
148110,2f083779-ac08-4708-8457-02c58725947d,02-05-2021,10.0,45,,551644,139,PHC Vanakbara,Daman and Diu,Diu,Diu,20.0,70.0,09:00:00,17:00:00,Free,139,139,Diu,37
148111,6c2dcbda-4f77-4449-b908-ee26922157ba,03-05-2021,10.0,45,,551644,139,PHC Vanakbara,Daman and Diu,Diu,Diu,20.0,70.0,09:00:00,17:00:00,Free,139,139,Diu,37
148112,5602f122-498d-4482-a0af-266e75f07f85,04-05-2021,10.0,45,,551644,139,PHC Vanakbara,Daman and Diu,Diu,Diu,20.0,70.0,09:00:00,17:00:00,Free,139,139,Diu,37


In [183]:
# Top 5 States Running ahead of everyone else.
session_center_district_df[['date', 'state_name']].groupby('state_name').apply(lambda x : x.sort_values(by = 'date', ascending = True).head(3).reset_index(drop = True))

date                   state_name
state_name                                                            
Andaman and Nicobar Islands 0  01-05-2021  Andaman and Nicobar Islands
                            1  01-05-2021  Andaman and Nicobar Islands
                            2  01-05-2021  Andaman and Nicobar Islands
Andhra Pradesh              0  01-05-2021               Andhra Pradesh
                            1  01-05-2021               Andhra Pradesh
...                                   ...                          ...
Uttarakhand                 1  01-05-2021                  Uttarakhand
                            2  01-05-2021                  Uttarakhand
West Bengal                 0  01-05-2021                  West Bengal
                            1  01-05-2021                  West Bengal
                            2  01-05-2021                  West Bengal

[111 rows x 2 columns]

In [204]:
session_center_district_df\
    [['date', 'state_name']]\
    .groupby('state_name')\
    .apply(lambda x : x.sort_values(by = 'date', ascending = True)\
    .head(1)\
    .reset_index(drop = True)).head(40)

,,date,state_name
state_name,,,
Andaman and Nicobar Islands,0,01-05-2021,Andaman and Nicobar Islands
Andhra Pradesh,0,01-05-2021,Andhra Pradesh
Arunachal Pradesh,0,01-05-2021,Arunachal Pradesh
Assam,0,01-05-2021,Assam
Bihar,0,01-05-2021,Bihar
Chandigarh,0,01-05-2021,Chandigarh
Chhattisgarh,0,01-05-2021,Chhattisgarh
Dadra and Nagar Haveli,0,01-05-2021,Dadra and Nagar Haveli
Daman and Diu,0,01-05-2021,Daman and Diu


In [203]:
session_center_district_df\
    [session_center_district_df["min_age_limit"]<45]\
    [session_center_district_df["fee_type"]=="Free"]\
    [['date', 'state_name']]\
    .groupby('state_name')\
    .apply(lambda x : x.sort_values(by = 'date', ascending = True)\
    .head(1)\
    .reset_index(drop = True)).head(40)

<ipython-input-203-2df4eb2f42ac>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  session_center_district_df\


,,date,state_name
state_name,,,
Andhra Pradesh,0,01-05-2021,Andhra Pradesh
Assam,0,01-05-2021,Assam
Bihar,0,01-05-2021,Bihar
Chhattisgarh,0,01-05-2021,Chhattisgarh
Delhi,0,01-05-2021,Delhi
Gujarat,0,01-05-2021,Gujarat
Haryana,0,01-05-2021,Haryana
Himachal Pradesh,0,01-05-2021,Himachal Pradesh
Jammu and Kashmir,0,01-05-2021,Jammu and Kashmir
